# Spectral Clustering as a Classification Method for Gut Microbiome Data

In [1]:
#importing libraries 
import pandas as pd
from scipy.spatial import distance
from utils_preprocessing import  merge_dataset
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import accuracy_score

In [2]:
dataset=merge_dataset(path_otu="data/curatedMD_diseased_train/20221207_diseased_taxrelabund_train.csv", path_metadata="data/curatedMD_diseased_train/20221207_diseased_metadata_train_18_columns.csv", level=5)

In [3]:
dataset.head()

,sample_id,study_name,subject_id,body_site,antibiotics_current_use,study_condition,disease,age,age_category,country,...,f__Sutterellaceae,f__Synergistaceae,f__Tannerellaceae,f__Thermaceae,f__Tissierellia_unclassified,f__Veillonellaceae,f__Vibrionaceae,f__Victivallaceae,f__Xanthomonadaceae,f__Yersiniaceae
0,SID31004,FengQ_2015,SID31004,stool,missing,CRC,CRC;fatty_liver;hypertension,64,adult,AUT,...,0.0,0.00000,0.00000,0.0,0.0,0.13020,0.0,0.0,0.0,0.0
1,SID31030,FengQ_2015,SID31030,stool,missing,adenoma,adenoma;fatty_liver;hypertension,70,senior,AUT,...,0.0,0.00000,0.28971,0.0,0.0,0.18229,0.0,0.0,0.0,0.0
2,SID31159,FengQ_2015,SID31159,stool,missing,CRC,CRC,73,senior,AUT,...,0.0,0.00000,0.45937,0.0,0.0,1.14717,0.0,0.0,0.0,0.0
3,SID31223,FengQ_2015,SID31223,stool,missing,CRC,CRC,65,adult,AUT,...,0.0,0.05291,0.32816,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
4,SID31237,FengQ_2015,SID31237,stool,missing,CRC,CRC;fatty_liver;hypertension,67,senior,AUT,...,0.0,0.00000,0.02137,0.0,0.0,0.00266,0.0,0.0,0.0,0.0


In [4]:
dataset['study_condition'].value_counts()

study_condition
IBD        654
CRC        555
IGT        233
T2D        198
adenoma    171
ACVD       171
Name: count, dtype: int64

In [5]:
ibd_class = dataset[dataset['study_condition'] == 'IBD']
crc_class = dataset[dataset['study_condition'] == 'CRC']

# Reduce the 'IBD' class to 555 samples
ibd_class_subset = ibd_class.sample(n=555, random_state=42)  # Adjust 

# Combine the subsets of both classes
final_dataset = pd.concat([ibd_class_subset, crc_class])

# Shuffle the final dataset to ensure randomness
final_dataset = final_dataset.sample(frac=1, random_state=42)

In [6]:
final_dataset

,sample_id,study_name,subject_id,body_site,antibiotics_current_use,study_condition,disease,age,age_category,country,...,f__Sutterellaceae,f__Synergistaceae,f__Tannerellaceae,f__Thermaceae,f__Tissierellia_unclassified,f__Veillonellaceae,f__Vibrionaceae,f__Victivallaceae,f__Xanthomonadaceae,f__Yersiniaceae
1697,SAMD00114897,YachidaS_2019,sub_10670,stool,missing,CRC,CRC,69,senior,JPN,...,0.00000,0.0,0.38378,0.0,0.0,29.50806,0.0,0.0,0.0,0.0
1224,V1_UC26_4,NielsenHB_2014,V1_UC26,stool,missing,IBD,IBD,36,adult,ESP,...,0.00000,0.0,1.89772,0.0,0.0,0.91117,0.0,0.0,0.0,0.0
355,CSM7KOTC,HMP_2019_ibdmdb,C3023,stool,no,IBD,IBD,60,adult,USA,...,0.62574,0.0,0.00000,0.0,0.0,0.72030,0.0,0.0,0.0,0.0
1480,mix35_mix36-N712-S507_GTAGAGGA-AAGGAGTA,ThomasAM_2019_c,mix35_mix36-N712-S507_GTAGAGGA-AAGGAGTA,stool,missing,CRC,CRC,28,adult,JPN,...,0.00000,0.0,2.58337,0.0,0.0,0.28302,0.0,0.0,0.0,0.0
1507,MMRS42780924ST-27-0-0,VogtmannE_2016,MMRS42780924ST-27-0-0,stool,missing,CRC,CRC,65,adult,USA,...,0.01482,0.0,2.86593,0.0,0.0,0.00000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,O2_UC37_2,NielsenHB_2014,O2_UC37,stool,missing,IBD,IBD,44,adult,ESP,...,0.83215,0.0,0.56393,0.0,0.0,8.86506,0.0,0.0,0.0,0.0
200,CSM67UFV,HMP_2019_ibdmdb,C3006,stool,no,IBD,IBD,32,adult,USA,...,0.00000,0.0,0.36014,0.0,0.0,0.00615,0.0,0.0,0.0,0.0
1884,SZAXPI017456-23,YuJ_2015,SZAXPI017456-23,stool,missing,CRC,CRC,77,senior,CHN,...,0.00000,0.0,0.10567,0.0,0.0,0.45300,0.0,0.0,0.0,0.0
1959,CCIS72607085ST-4-0,ZellerG_2014,FR-824,stool,missing,CRC,CRC,74,senior,FRA,...,0.00000,0.0,5.88908,0.0,0.0,4.27232,0.0,0.0,0.0,0.0


In [7]:
final_dataset.loc[:, 'class'] = final_dataset['study_condition']

In [8]:
final_dataset=final_dataset.iloc[:,18:]

In [9]:
final_dataset=final_dataset.reset_index(drop=True)

In [10]:
final_dataset['class'].value_counts()

class
CRC    555
IBD    555
Name: count, dtype: int64

In [11]:
final_dataset.to_csv("microbiome_family_level.csv", index=False)

In [12]:
X=final_dataset.drop(columns=['class'])
y=final_dataset['class']

In [13]:
# TODO split the dataset X and y to create an hold hout which is gonna be used to test the final model with the optimal thresholds

## Spectral Clustering Analysis

In [23]:
def get_distance_matrix(dataset, metric="braycurtis"):
    distance_matrix=distance.squareform(distance.pdist(dataset, metric=metric))
    #avoiding NaN values
    distance_matrix=pd.DataFrame(distance_matrix).fillna(0).values
    return distance_matrix

def get_accuracy(cluster_labels: np.ndarray, y_train: pd.core.series.Series)->int:
  mapping_class_to_clusters=pd.DataFrame({'cluster':list(cluster_labels), 'class':list(y_train)})
  # given this dataset we need to determine what is the majority class for each cluster
  group_by_class_to_clusters=mapping_class_to_clusters.groupby(by=["cluster", "class"]).size().reset_index(name="count")
  max_counts_idx = group_by_class_to_clusters.groupby('cluster')['count'].idxmax()
  
  # Display the corresponding rows
  result = group_by_class_to_clusters.loc[max_counts_idx]
  total_number_of_samples=group_by_class_to_clusters['count'].sum()
  correctly_predicted=result['count'].sum()
  accuracy=correctly_predicted/total_number_of_samples
  accuracy=accuracy.round(3)
  return accuracy,result

def get_accuracy_test(X_train_graph_similarity, X_test_graph_similarity,cluster_labels, y_test, result_classes_per_cluster):
  clf=fit_nearest_centroid(X_train_graph_similarity,cluster_labels)
  y_test_predicted_cluster=clf.predict(X_test_graph_similarity)
  clusters_list=result_classes_per_cluster['cluster'].to_list()
  classes_list=result_classes_per_cluster['class'].to_list()
  mapping_cluster_to_class_dict={clusters_list[i]:classes_list[i] for i in range(len(clusters_list))}
  y_test_predicted_class=[mapping_cluster_to_class_dict[p] for p in y_test_predicted_cluster]
  #get the accuracy
  accuracy_test=accuracy_score(y_test,y_test_predicted_class)
  accuracy_test=accuracy_test.round(3)
  return accuracy_test

def fit_nearest_centroid(X_train_graph_similarity, cluster_labels):
  clf = NearestCentroid()
  clf.fit(X_train_graph_similarity,cluster_labels)
  return clf

def get_test_set_distances(X_test,X_train):
  X_train_array = X_train.values
  X_test_array = X_test.values
  # Calculate Bray-Curtis distance using cdist
  result_matrix = distance.cdist(X_test_array, X_train_array, metric='braycurtis')
  #avoiding NaN values
  distance_matrix=pd.DataFrame(result_matrix).fillna(0).values
  return distance_matrix

def iterated_hold_out_clustering(X:pd.core.frame.DataFrame,y:pd.core.series.Series,iterations:int,thresh:float)->list:
  results={'training_accuracy':[], 'test_accuracy':[]}
  for i in range(iterations):
    # splitting dataset in training and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=i)
    X_train_graph=get_distance_matrix(X_train)
    X_train_graph[X_train_graph>=thresh]=1
    X_train_graph_similarity=1-X_train_graph
    clusterer = SpectralClustering(n_clusters=2, random_state=i, affinity="precomputed", n_jobs=-1)
    # distance matrix is computed using braycurtis distance!
    cluster_labels = clusterer.fit_predict(X_train_graph_similarity)
    accuracy_training,result_classes_per_cluster=get_accuracy(cluster_labels,y_train)
    results['training_accuracy'].append(accuracy_training)
    # testing the model
    X_test_graph=get_test_set_distances(X_test,X_train) 
    #use the threshold
    X_test_graph[X_test_graph>=thresh]=1
    #convert distance to similarity
    X_test_graph_similarity=1-X_test_graph
    #get test accuracy
    test_accuracy=get_accuracy_test(X_train_graph_similarity, X_test_graph_similarity, cluster_labels, y_test, result_classes_per_cluster)
    results['test_accuracy'].append(test_accuracy)

  return results

def grid_search_threshold(X:pd.core.frame.DataFrame, y:pd.core.series.Series, thresholds:list, hold_out_it:int)->dict:
  results_gridsearch={'thresh':[], 'training_accuracy':[], 'test_accuracy':[]}
  
  for t in thresholds:
    results_gridsearch['thresh'].append(t)
    results_hold_out=iterated_hold_out_clustering(X,y,hold_out_it,t)
    mean_training=np.mean(np.asarray(results_hold_out['training_accuracy']))
    mean_test=np.mean(np.asarray(results_hold_out['test_accuracy']))
    results_gridsearch['training_accuracy'].append(mean_training)
    results_gridsearch['test_accuracy'].append(mean_test)
    
  return results_gridsearch


In [39]:
pd.DataFrame(grid_search_threshold(X,y,[0.975,0.6],2))

,thresh,training_accuracy,test_accuracy
0,0.975,0.6480,0.650
1,0.600,0.5205,0.458


In [18]:
'''i=10
thresh=0.975
results={'training_accuracy':[], 'test_accuracy':[]}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=i)
X_train_graph=get_distance_matrix(X_train)
X_train_graph[X_train_graph>=thresh]=1
X_train_graph_similarity=1-X_train_graph
clusterer = SpectralClustering(n_clusters=2, random_state=i, affinity="precomputed", n_jobs=-1)
# distance matrix is computed using braycurtis distance!
cluster_labels = clusterer.fit_predict(X_train_graph_similarity)
accuracy_training,result_classes_per_cluster=get_accuracy(cluster_labels,y_train)
results['training_accuracy'].append(accuracy_training)
# doing the same procedure on X_test
X_test_graph=get_test_set_distances(X_test,X_train) #use get_test_set_distances
#use the threshold
X_test_graph[X_test_graph>=thresh]=1
#convert distance to similarity
X_test_graph_similarity=1-X_test_graph
#get test accuracy
test_accuracy=get_accuracy_test(X_train_graph_similarity, X_test_graph_similarity, cluster_labels, y_test, result_classes_per_cluster)
results['test_accuracy'].append(test_accuracy)
'''

In [ ]:
# TODO produce all the charts to show how the performance vary with different values

In [ ]:
# Use the best parameters to test the final clustering 


In [145]:
X_train

,f__Acidaminococcaceae,f__Actinomycetaceae,f__Aerococcaceae,f__Aeromonadaceae,f__Akkermansiaceae,f__Alcaligenaceae,f__Anaerolineaceae,f__Atopobiaceae,f__Bacillaceae,f__Bacillales_unclassified,...,f__Sutterellaceae,f__Synergistaceae,f__Tannerellaceae,f__Thermaceae,f__Tissierellia_unclassified,f__Veillonellaceae,f__Vibrionaceae,f__Victivallaceae,f__Xanthomonadaceae,f__Yersiniaceae
350,0.04993,0.07841,0.0,0.0,12.81751,0.0,0.0,0.01341,0.0,0.20511,...,0.00000,0.00000,1.84399,0.0,0.0,0.08781,0.0,0.00000,0.0,0.00000
800,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.00000,...,0.00468,0.00000,2.72978,0.0,0.0,1.22646,0.0,0.00000,0.0,0.00000
5,1.04715,0.00000,0.0,0.0,3.76740,0.0,0.0,0.00000,0.0,0.00000,...,0.00000,0.13811,2.13486,0.0,0.0,2.97837,0.0,0.00000,0.0,0.00000
310,0.00000,0.00000,0.0,0.0,1.28008,0.0,0.0,0.00000,0.0,0.00000,...,0.00000,0.00000,2.09387,0.0,0.0,0.38273,0.0,0.00000,0.0,0.00000
552,0.00000,0.19912,0.0,0.0,0.10730,0.0,0.0,0.89726,0.0,0.02199,...,0.00000,0.00000,3.32243,0.0,0.0,4.54348,0.0,0.00000,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.00000,...,0.00000,0.00000,13.93696,0.0,0.0,0.07740,0.0,0.00000,0.0,0.00000
121,0.00098,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.00000,...,0.00000,0.00000,5.40373,0.0,0.0,0.00000,0.0,0.00000,0.0,0.00000
1044,2.16107,0.03976,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.04732,...,0.02147,0.00722,7.52014,0.0,0.0,0.01185,0.0,0.00000,0.0,0.00000
1095,7.08993,0.00000,0.0,0.0,0.00657,0.0,0.0,0.00000,0.0,0.00000,...,0.00000,0.00000,3.01400,0.0,0.0,1.33212,0.0,0.00000,0.0,0.23903


In [144]:
X_test

,f__Acidaminococcaceae,f__Actinomycetaceae,f__Aerococcaceae,f__Aeromonadaceae,f__Akkermansiaceae,f__Alcaligenaceae,f__Anaerolineaceae,f__Atopobiaceae,f__Bacillaceae,f__Bacillales_unclassified,...,f__Sutterellaceae,f__Synergistaceae,f__Tannerellaceae,f__Thermaceae,f__Tissierellia_unclassified,f__Veillonellaceae,f__Vibrionaceae,f__Victivallaceae,f__Xanthomonadaceae,f__Yersiniaceae
884,3.47800,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.00397,...,0.00418,0.00000,0.69748,0.0,0.0,0.01211,0.0,0.00000,0.0,0.0
342,1.66718,0.00000,0.0,0.0,0.02727,0.0,0.0,0.00000,0.0,0.00000,...,0.09935,0.02088,4.16965,0.0,0.0,0.21201,0.0,0.00000,0.0,0.0
56,0.21352,0.00000,0.0,0.0,0.26190,0.0,0.0,0.00000,0.0,0.00000,...,0.61262,0.00000,0.98445,0.0,0.0,1.49301,0.0,0.00000,0.0,0.0
694,0.00236,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.00000,...,0.00000,0.00000,5.87853,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
721,2.04091,0.00000,0.0,0.0,0.01273,0.0,0.0,0.00000,0.0,0.00000,...,0.00000,0.00000,3.19114,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
888,4.62272,0.01118,0.0,0.0,0.03734,0.0,0.0,0.00151,0.0,0.04697,...,0.00000,0.05173,2.13850,0.0,0.0,0.02479,0.0,0.01778,0.0,0.0
882,1.31878,0.00202,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.00000,...,0.00000,0.00000,3.80362,0.0,0.0,0.01689,0.0,0.00000,0.0,0.0
881,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.00000,...,0.00000,0.00000,0.00000,0.0,0.0,0.28995,0.0,0.00000,0.0,0.0
432,0.05365,0.00000,0.0,0.0,0.00000,0.0,0.0,0.00000,0.0,0.00000,...,0.05153,0.00000,8.14119,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0


In [ ]:
distance.pdist([[X_test]], metric="braycurtis")

In [148]:
len(X_train_graph[0])

743

In [153]:
X_train_array = X_train.values
X_test_array = X_test.values

# Calculate Bray-Curtis distance using cdist
result_matrix = distance.cdist(X_test_array, X_train_array, metric='braycurtis')

In [156]:
len(result_matrix[0])

743

In [162]:
len(X_test_array)

367

In [161]:
X_test.values[0]

99

In [150]:
X_test[:, np.newaxis, :]

InvalidIndexError: (slice(None, None, None), None, slice(None, None, None))